In [1]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params

2023-09-08 00:17:12 Didis-MacBook-Pro.local metapub.config[56608] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [4]:
# download pdf to specified folder given pdf_url and ind
def download_pdf(pdf_url, pdf_folder):  
    url = pdf_url
    if url != url:
        raise Exception("pdf_url is np.nan")
    else:
        source = url.split("://")[1].split("/")[0]
    
    # pdf_path = os.path.join(pdf_folder, file_name)
      
    # get the pdf content
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option('prefs', {
    # "download.default_directory": pdf_folder, #Change default directory for downloads
    # "download.prompt_for_download": False, #To auto download the file
    # "download.directory_upgrade": True,
    # "download.open_pdf_in_system_reader": False,
    # "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    # })
    # driver = webdriver.Chrome(options=options)
    # # response = driver.get(pdf_url)
    # driver.navigate().to(url)
    # time.sleep(5)
    if source == "www.sciencedirect.com":
        os.environ['WDM_LOG'] = '0'
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome()
        driver.get(url)
        element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='ViewPDF']/a")))
        element.click()
        time.sleep(10)
    else: 
        options = webdriver.ChromeOptions()
        options.add_experimental_option('prefs', {
        "download.default_directory": pdf_folder, #Change default directory for downloads
        "download.prompt_for_download": False, #To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
        })
        driver = webdriver.Chrome(options=options)
        response = driver.get(url)
        time.sleep(10)
    driver.close()

    # try:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         driver.close()
    #         return True
    # except:
    #     print(f'Failed downloading PDF:', url)
    #     driver.close()
    #     # print(f'HTTP response status code: {response.status_code}')
    #     return False
# --------------------start of test code--------------------
# # pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# # pdf_url = "https://www.sciencedirect.com/science/article/pii/S1053811909013299?via%3Dihub"
# # pdf_url = "https://linkinghub.elsevier.com/retrieve/pii/S1053811909013299"
# # pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1181753/pdf/jphysiol00456-0559.pdf"
# pdf_url = "https://www.sciencedirect.com/science/article/pii/S0896627320300052?via%3Dihub"

# # wiley.com
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902970309"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902970309"

# # physiology.org
# # pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"

# # springer.com
# # pdf_url = "https://link.springer.com/content/pdf/10.1007/BF00236173.pdf"

# # cell.com
# # pdf_url = "https://www.cell.com/neuron/pdf/S0896-6273(09)00170-6.pdf"

# # jneurosci.org
# # pdf_url = "https://www.jneurosci.org/content/jneuro/22/18/8117.full.pdf"
# # pdf_url, code = plib.get_final_redirected_url(pdf_url)
# # print(pdf_url)
# ind = 10
# file_name = str(ind) + ".pdf"
# pdf_folder = fpath.litera_pdf_folder
# download_pdf(pdf_url, pdf_folder)
# # rename_pdf(file_name, pdf_folder, time_to_wait=60)
# ---------------------end of test code---------------------

In [5]:
def download_from_www_ncbi_nlm_nih_gov(url, ind, pdf_folder):
    try:
        file_name = str(ind) + ".pdf"
        response = requests.get(url, headers=plib.headers)
        # response = requests.get(url, stream=True, allow_redirects=True, headers=plib.headers)
        
        # download the .pdf file to the pdf_file_path folder
        # write content in pdf file
        pdf_path = os.path.join(pdf_folder, file_name)
        
        if response.status_code == 200:
            with open(pdf_path, 'wb') as pdf_object:
                pdf_object.write(response.content)
            print(f'Successfully downloaded PDF:', ind)
        else:
            print(f'Failed downloading PDF:' + 'pdf_url')
            print(f'HTTP response status code: {response.status_code}')
    except:
        print(f'Failed downloading PDF:' + 'pdf_url')
# --------------------start of test code--------------------
# pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6577493/pdf/jneuro_14_5_2485.pdf"
# ind = 10
# file_name = str(ind) + ".pdf"
# pdf_folder = "/home/hou/myProjects/litera_pdfs"
# download_from_www_ncbi_nlm_nih_gov(pdf_url, ind, pdf_folder)
# # rename_pdf(file_name, pdf_folder, time_to_wait=60)
# ---------------------end of test code---------------------

In [6]:
# def download_from_ELSEVIER(doi, file_to_save_to):
#     url = 'http://api.elsevier.com/content/article/doi:'+doi+'?view=FULL'
#     headers = {
#         'X-ELS-APIKEY': "63f58b8b10cbc1bc923011c01c6301bb",
#         'Accept': 'application/pdf'
#     }
#     r = requests.get(url, stream=True, headers=headers)
#     if r.status_code == 200:
#         for chunk in r.iter_content(chunk_size=1024*1024):
#             file_to_save_to.write(chunk)
#         return True

# # doi_list = pd.read_excel('list.xls')
# # doi_list.columns = ['DOIs']
# count = 0
# # for doi in doi_list['DOIs']:
# doi = "10.1016/0006-8993(95)01338-5"
# # pdf = "10"
# ind = 10
# file_name = str(ind) + ".pdf"
# # if not os.path.exists(f'path/{pdf}.pdf'):
# with open(file_name, 'wb') as pdf_object:
#     get_pdf(doi, pdf_object)
# pdf_object.close

In [7]:
# download pdf
def download_pdf(pdf_url, pdf_source, pdf_folder):
    for website in params.website_pdfs:
        func = None
        if website in pdf_source:
            # Get the function name by replacing "." with "_" and use globals() to call it
            func_name = "download_from_" + website.replace(".", "_")
            # print(func_name)
            func = globals().get(func_name)
            # print(func)
            break
    if func != None:
        # print(func)
        func(pdf_url, pdf_folder)
    else:
        print("The given url is not from a supported website: ", pdf_url)
        raise Exception("Function does not exist for website:", pdf_url)

# rename downloaded pdf
def rename_pdf(ind, pdf_folder, time_to_wait=60):
    newname = str(ind) + ".pdf"
    time_counter = 0
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    while '.part' in filename:
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa :   os.path.getctime(os.path.join(pdf_folder,xa)))
    os.rename(os.path.join(pdf_folder, filename), os.path.join(pdf_folder, newname))

# download and rename pdf
def download_and_rename_pdf(pdf_url, ind, pdf_folder):  
    if pdf_url != pdf_url:
        raise Exception("pdf_url is np.nan")
    else:
        pdf_source = pdf_url.split("://")[1].split("/")[0]
    download_pdf(pdf_url, pdf_source, pdf_folder)
    rename_pdf(ind, pdf_folder, time_to_wait=60)
    print(f"Downloaded {ind}.pdf")
# --------------------start of test code--------------------
# pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"
# ind = 10
# pdf_folder = "/home/hou/myProjects/litera_pdfs"
# download_and_rename_pdf(pdf_url, ind, pdf_folder)
# ---------------------end of test code---------------------
# https://journals.physiology.org/doi/10.1152/jn.2001.85.1.219

In [8]:
# full_text_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5856726/'
# doi = '10.1007/s12264-018-0210-1'
# pmid = '29430586'
# pmcid = 'PMC5856726'
# if full_text_url == full_text_url and full_text_url.split("://")[1].split("/")[0] == 'www.ncbi.nlm.nih.gov':
#     if doi == doi:
#         url = "https://doi.org/" + doi
#         url, status_code = plib.get_final_redirected_url(url)
#         info = extract_info.extract_info_from_webpage(url, params.websites)
#         full_text_url = url
#         full_text_source = url.split("://")[1].split("/")[0]
#         pdf_url = info["pdf_link"]
#         pdf_source = pdf_url.split("://")[1].split("/")[0]
#     else:
#         pdf_url = np.nan
#         pdf_source = np.nan
# else:  
#     print("PDF_URL not found for: ", doi, pmid, pmcid, full_text_url)
#     pdf_url = np.nan
#     pdf_source = np.nan
# print(doi)
# print(pmid)
# print(pmcid)
# print(full_text_url)
# print(full_text_source)
# print(pdf_url)
# print(pdf_source)